In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
# Imports 

import matplotlib.pyplot as plt
import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import plot_confusion_matrix

In [ ]:
#Load csv file

data = pd.read_csv("/kaggle/input/zelpactivitytrainassignment/train_imu.csv")
test_data = pd.read_csv('../input/zelpactivitytrainassignment/train_imu.csv')


data

In [ ]:
test_data

In [ ]:
# Explore dataset

print(data['labels'].nunique(), "Classes:")
print(data['labels'].value_counts())


# This query shows the dataset to have significantly unbalanced class sizes

In [ ]:
# Convert index to 'datetime' column and 'datetime' type String -> Datetime
data.index = pd.to_datetime(data.index, unit='s')
test_data.index = pd.to_datetime(test_data.index, unit='s')

In [ ]:
#Resample IMU readings to mean value per Second
#so that each second is represented by a single sample

standardised_data = data.groupby('datetime').mean()
standardised_test_data = test_data.groupby('datetime').mean()



#standardised_data = data.mean()
#standardised_test_data = test_data.mean()

standardised_test_data

In [ ]:

print(standardised_test_data.head(10))

In [ ]:
# Plot a given feature

import matplotlib.pyplot as plt

def visualise_feature(feature, n_samples):
    data_sample = standardised_data[:n_samples]
    plt.plot(data_sample[feature])
    print(feature, "change over", n_samples, "seconds")
    
    
    
visualise_feature('Gy', 400)

In [ ]:
#Seperate features & labels

y = standardised_data['labels']
x = standardised_data.drop('labels', 1)

# Test, train split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)



In [ ]:
# Define classifier
clf = RandomForestClassifier(n_estimators = 100, verbose = 2,
                             class_weight = 'balanced') # balanced classes to mitigate effect of class sizes


In [ ]:
#Train Classifier
clf.fit(x,y)


In [ ]:
# Explore confusion matrix
#print(plot_confusion_matrix(clf, x_test, y_test, cmap = 'Greys', normalize = 'true'))

In [ ]:
# Get Predictions on test set
standardised_test_data = standardised_test_data.drop('labels', 1)

predictions = clf.predict(standardised_test_data)

In [ ]:
#Define Output
output = standardised_test_data

#Append model predictions to output df
output['labels'] = predictions

# Remove feature columbs
output = output['labels']


#show number of labels
print(output.nunique(), "Classes:")
print(output.value_counts())


In [ ]:
test = output.to_frame()
test = test.reset_index()

In [ ]:
#test['datetime'].nunique()
test.tail(10)

In [ ]:
output_csv = test.to_csv('/kaggle/working/test.csv', index=False)

In [ ]:
output_csv